In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：用于在线预测的AutoML训练图像对象检测模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_online.ipynb">
      在Google Cloud笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述


本教程演示如何使用Vertex SDK创建图像对象检测模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行在线预测。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的沙拉类别的[OpenImages数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)。这个数据集不需要任何特征工程。在本教程中使用的数据集版本存储在公共云存储桶中。训练好的模型可以预测图像中五种类型的沙拉成分（沙拉、海鲜、番茄、烘焙食品或奶酪）的边界框位置和对应类型。

### 目标

在本教程中，您将使用 Vertex SDK 从Python脚本中创建一个AutoML图像目标检测模型，并部署到在线预测。您也可以使用`gcloud`命令行工具或在Cloud Console上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个serving `Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)根据您预计的使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，则您的环境已经满足运行此笔记本的所有要求。 您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。 您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的云存储指南](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。 以下步骤提供了一套简洁的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。 激活虚拟环境。

4. 要安装Jupyter，在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###GPU运行时

这个教程不需要GPU运行时。

###设置你的Google Cloud项目

**无论你的笔记本环境是什么，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账号时，您将获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs，Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您将需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入你的项目ID。然后运行单元格以确保Cloud SDK在本笔记本的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行视为shell命令，并插入以`$`为前缀的Python变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您也可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是 Vertex AI 支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不能使用多区域存储桶进行 Vertex AI 的训练。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会话，可能会使用共享测试账户或项目。为避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将该时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经经过认证。请跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行操作来通过oAuth认证您的账户。

**否则**，请按照以下步骤进行：

在云控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务账户访问项目**部分，点击角色下拉菜单。在过滤框中键入"Vertex"并选择**Vertex管理员**。在过滤框中键入"存储对象管理员"并选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境。

将您的服务账户密钥路径输入为下面单元格中的GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要按照以下步骤操作。**

当您初始化Python的Vertex SDK时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关数据在各个会话中保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

建立变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 初始化 Vertex SDK for Python

为您的项目和对应的存储桶初始化 Vertex SDK for Python。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您已经准备好开始创建自己的AutoML图像目标检测模型。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的一个Salads数据集版本，其中使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标注模式。

此操作可能需要几分钟的时间。

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="Salads" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行管道。

#### 创建训练管道

使用`AutoMLImageTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：用于训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则是单标签(`False`)还是多标签(`True`)。
- `model_type`：部署模型的类型。
  - `CLOUD`：在Google Cloud上部署。
  - `CLOUD_HIGH_ACCURACY_1`：针对Google Cloud部署的准确性优化。
  - `CLOUD_LOW_LATENCY_`：针对Google Cloud部署的延迟优化。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：针对边缘设备的准确性优化。
  - `MOBILE_TF_LOW_LATENCY_1`：针对边缘设备的延迟优化。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习--仅支持图像分类。

实例化的对象是训练作业的有向无环图(DAG)。

In [ ]:
dag = aip.AutoMLImageTrainingJob(
    display_name="salads_" + TIMESTAMP,
    prediction_type="object_detection",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

#### 运行训练流水线

接下来，您可以通过调用方法 `run` 来运行DAG，以启动训练工作，并提供以下参数：

- `dataset`：用于训练模型的 `Dataset` 资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果设为 `True`，则服务可能认为无法进一步改善模型目标指标之前已经完成训练。

完成`run`方法后会返回 `Model` 资源。

训练流水线的执行将需要最多60分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="salads_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

## 回顾模型评估分数
在您的模型训练完成后，您可以查看它的评估分数。

首先，您需要获取新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=salads_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

部署模型

接下来，部署你的模型用于在线预测。要部署模型，你需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

发送一个在线预测请求

向您部署的模型发送一个在线预测。

获取测试项目

您将从数据集中任意选择一个示例作为测试项目。不用担心这个示例很可能在训练模型时被使用过 - 我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n1
cols = str(test_items[0]).split(",")
if len(cols) == 11:
    test_item = str(cols[1])
    test_label = str(cols[2])
else:
    test_item = str(cols[0])
    test_label = str(cols[1])

print(test_item, test_label)

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源，您可以通过向 Endpoint 资源发送预测请求来进行在线预测。

#### 请求

在这个例子中，由于您的测试项目位于一个 Cloud Storage 存储桶中，您打开并读取图像的内容，使用 `tf.io.gfile.Gfile()`。为了将测试数据传递到预测服务，您需要将字节编码为 base64 -- 这样可以在通过网络传输二进制数据时确保内容不被修改。

每个实例的格式为：

    { 'content': { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项目（实例），所以将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

`predict()` 调用返回的是一个 Python 字典，其中包含以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类别标签的类名。
- `confidences`: 每个类别标签的预测置信度，值在0到1之间。
- `bboxes`: 每个检测到的对象的边界框。
- `deployed_model_id`: 执行预测的 Vertex AI 资源所部署模型的标识符。

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

取消部署模型

当你完成了预测之后，你可以从`Endpoint`资源中取消部署模型。这将取消所有计算资源，并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME